In [1]:
import pandas as pd
import numpy as np

# --- 自動化對帳機器人 ---

# 1. 模擬資料
data_internal = {
    'OrderID': ['A001', 'A002', 'A003', 'A004', 'A005'],
    'Amount': [1000, 2000, 3000, 4000, 5000],
}
df_internal = pd.DataFrame(data_internal)

data_bank = {
    'OrderID': ['A001', 'A003', 'A004', 'A005', 'A006'],
    'Amount': [1000, 2990, 4000, 5000, 6000], 
}
df_bank = pd.DataFrame(data_bank)

# 2. 合併比對 (Outer Join)
merged_df = pd.merge(df_internal, df_bank, on='OrderID', how='outer', suffixes=('_Co', '_Bank'), indicator=True)

# 3. 邏輯判斷
status_list = []
for index, row in merged_df.iterrows():
    if row['_merge'] == 'left_only':
        status_list.append('異常：漏帳 (公司有, 銀行無)')
    elif row['_merge'] == 'right_only':
        status_list.append('異常：溢帳 (銀行有, 公司無)')
    else:
        if row['Amount_Co'] == row['Amount_Bank']:
            status_list.append('正常')
        else:
            diff = row['Amount_Co'] - row['Amount_Bank']
            status_list.append(f'異常：金額不符 (差額 {diff})')

merged_df['Recon_Status'] = status_list

# 4. 產出差異報告
diff_report = merged_df[merged_df['Recon_Status'] != '正常'].copy()
diff_report[['OrderID', 'Amount_Co', 'Amount_Bank', 'Recon_Status']]

,OrderID,Amount_Co,Amount_Bank,Recon_Status
1,A002,2000.0,NaN,"異常：漏帳 (公司有, 銀行無)"
2,A003,3000.0,2990.0,異常：金額不符 (差額 10.0)
5,A006,NaN,6000.0,"異常：溢帳 (銀行有, 公司無)"
